##Task 3 — Multimodal Housing Price Prediction (Images + Tabular)

In [ ]:
# ====== STEP 1: Install & Imports ======
!pip install -q torch torchvision scikit-learn pandas numpy pillow joblib matplotlib

import os, pandas as pd, numpy as np, torch, joblib
from PIL import Image
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from google.colab import files

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ====== STEP 2: Load CSV (upload if needed) ======
CSV_NAME = "houses.csv"  # change if different
if not os.path.exists(CSV_NAME):
    print("Upload your housing CSV (must include target price and image reference).")
    up = files.upload()
    CSV_NAME = list(up.keys())[0]

df = pd.read_csv(CSV_NAME)
print(df.head())

# ====== STEP 3: Identify Columns ======
# Target column (change if your target name differs)
TARGET = "price"
assert TARGET in df.columns, f"Expected target column '{TARGET}' not found."

# Image column: either 'image_path' or 'image_id'
IMG_COL = None
if "image_path" in df.columns:
    IMG_COL = "image_path"
elif "image_id" in df.columns:
    IMG_COL = "image_id"
else:
    raise ValueError("Expected 'image_path' or 'image_id' column to reference images.")

# Resolve actual file path from row (supports image_id -> images/<id>.jpg)
def resolve_image_path(row):
    if IMG_COL == "image_path":
        return row[IMG_COL]
    else:
        return f"images/{row[IMG_COL]}.jpg"

df["__img_path"] = df.apply(resolve_image_path, axis=1)

# Separate tabular features (exclude target + image refs)
tab_cols = [c for c in df.columns if c not in [TARGET, IMG_COL, "__img_path"]]
X_tab = df[tab_cols].copy()
y = df[TARGET].astype(float)

# Convert non-numeric tabular to numeric (one-hot)
X_tab = pd.get_dummies(X_tab, drop_first=True)

# ====== STEP 4: Image Feature Extractor (ResNet18) ======
resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet.fc = torch.nn.Identity()  # output 512-d features
resnet = resnet.to(device).eval()

img_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=models.ResNet18_Weights.DEFAULT.meta["mean"],
                         std=models.ResNet18_Weights.DEFAULT.meta["std"]),
])

@torch.no_grad()
def extract_feature(img_path: str):
    if not os.path.exists(img_path):
        # If image missing, return zeros
        return np.zeros(512, dtype=np.float32)
    img = Image.open(img_path).convert("RGB")
    t = img_tfms(img).unsqueeze(0).to(device)
    feat = resnet(t).squeeze(0).detach().cpu().numpy().astype(np.float32)
    return feat

# ====== STEP 5: Build Image Embeddings Matrix ======
img_feats = np.vstack([extract_feature(p) for p in df["__img_path"]])
print("Image feature matrix:", img_feats.shape)  # (N, 512)

# ====== STEP 6: Train/Test Split ======
X_train_tab, X_test_tab, X_train_img, X_test_img, y_train, y_test = train_test_split(
    X_tab, img_feats, y, test_size=0.2, random_state=42
)

# Scale tabular features only
scaler = StandardScaler(with_mean=False)  # sparse-safe
X_train_tab_scaled = scaler.fit_transform(X_train_tab)
X_test_tab_scaled  = scaler.transform(X_test_tab)

# Concatenate tabular + image features
from scipy.sparse import issparse, hstack, csr_matrix

if issparse(X_train_tab_scaled):
    X_train_combined = np.hstack([X_train_tab_scaled.toarray(), X_train_img])
    X_test_combined  = np.hstack([X_test_tab_scaled.toarray(),  X_test_img])
else:
    X_train_combined = np.hstack([X_train_tab_scaled, X_train_img])
    X_test_combined  = np.hstack([X_test_tab_scaled,  X_test_img])

print("Combined feature shapes:", X_train_combined.shape, X_test_combined.shape)

# ====== STEP 7: Train Regressor ======
reg = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
reg.fit(X_train_combined, y_train)

# ====== STEP 8: Evaluate ======
pred = reg.predict(X_test_combined)
mae = mean_absolute_error(y_test, pred)
rmse = mean_squared_error(y_test, pred, squared=False)
r2 = r2_score(y_test, pred)
print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f} | R²: {r2:.3f}")

plt.figure(figsize=(6,6))
plt.scatter(y_test, pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
plt.xlabel("Actual Price"); plt.ylabel("Predicted Price")
plt.title("Actual vs Predicted")
plt.show()

# ====== STEP 9: Save Model + Scaler + Metadata ======
joblib.dump(reg, "multimodal_housing_reg.joblib")
joblib.dump(scaler, "tabular_scaler.joblib")
pd.Series(tab_cols).to_csv("tabular_columns.csv", index=False)
print("Saved: multimodal_housing_reg.joblib, tabular_scaler.joblib, tabular_columns.csv")

# ====== STEP 10: Example Inference ======
def predict_price(row_idx=0):
    row_tab = X_test_tab.iloc[[row_idx]]
    row_img = X_test_img[[row_idx]]
    row_tab_scaled = scaler.transform(row_tab)
    if issparse(row_tab_scaled):
        x = np.hstack([row_tab_scaled.toarray(), row_img])
    else:
        x = np.hstack([row_tab_scaled, row_img])
    return float(reg.predict(x)[0])

print("Example prediction:", predict_price(0))



Upload your Telco churn CSV (e.g., Telco-Customer-Churn.csv)
